## Covering all model components
* mean model --> so far: Zero mean, try out ARIMA
* volatility model --> GARCH
* distribution --> normal distributed

Followed this article:
https://medium.com/analytics-vidhya/arima-garch-forecasting-with-python-7a3f797de3ff

In [24]:
from arch import arch_model
from datetime import timedelta
import pandas as pd
import numpy as np

from evaluation.help_functions.prepare_data import most_recent_thursday, next_working_days
from dax.help_functions.get_dax_data import get_prepared_data
from dax.help_functions.get_quantiles import get_norm_quantiles
from dax.help_functions.get_dax_data import get_data

In [25]:
daxdata = get_prepared_data()
daxdata.loc['2019-09-10 00:00:00+02:0':]

,Close,LogRetLag1,LogRetLag2,LogRetLag3,LogRetLag4,LogRetLag5
Date,,,,,,
2019-09-10 00:00:00+02:00,12268.709961,0.347914,0.629422,1.163586,2.006097,2.960150
2019-09-11 00:00:00+02:00,12359.070312,0.733812,1.081725,1.363234,1.897398,2.739909
2019-09-12 00:00:00+02:00,12410.250000,0.413251,1.147063,1.494977,1.776485,2.310649
2019-09-13 00:00:00+02:00,12468.530273,0.468515,0.881766,1.615578,1.963491,2.245000
2019-09-16 00:00:00+02:00,12380.309570,-0.710062,-0.241547,0.171704,0.905516,1.253429
...,...,...,...,...,...,...
2024-01-04 00:00:00+01:00,16617.289062,0.475928,-0.910971,-0.805254,-0.505791,-0.748107
2024-01-05 00:00:00+01:00,16594.210938,-0.138977,0.336951,-1.049948,-0.944231,-0.644768
2024-01-08 00:00:00+01:00,16716.470703,0.734061,0.595084,1.071012,-0.315887,-0.210170


LogRetLag1 ---> 1. Prediction at t
LogRegLag2 --> 2. Prediction at t
LogRetLag3 --> 3. Prediction at t

## Mean Component 
* Find ARIMA-order for each Horizon
* Use Library PMDARIMA, IC = AIC, BIC
* Unter beiden Kriterien folgende Order: 
{'LogRetLag1': (0, 0, 0),
 'LogRetLag2': (5, 0, 0),
 'LogRetLag3': (5, 0, 1),
 'LogRetLag4': (5, 0, 0),
 'LogRetLag5': (5, 0, 2)}


In [26]:
import yfinance as yf
from statsmodels.tsa.stattools import adfuller
import numpy as np

def test_stationarity(daxdata):
    # Berechnung der Log-Renditen

    # Durchführung des Augmented Dickey-Fuller-Tests
    result = adfuller(daxdata['LogRetLag1'], autolag='AIC')

    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    print('Critical Values:', result[4])

    # Überprüfung des p-Werts
    if result[1] <= 0.05:
        print("Die Zeitreihe der Log-Renditen ist stationär.")
    else:
        print("Die Zeitreihe der Log-Renditen ist nicht stationär.")

# Stationaritätstest durchführen
test_stationarity(daxdata)

ADF Statistic: -8.074532544987864
p-value: 1.5179612772993655e-12
Critical Values: {'1%': -3.436402509014354, '5%': -2.8642124318084456, '10%': -2.568192893555997}
Die Zeitreihe der Log-Renditen ist stationär.


In [27]:
import pmdarima
import statsmodels as sm

order_dict_aic = {}
for h in ['LogRetLag1', 'LogRetLag2', 'LogRetLag3', 'LogRetLag4','LogRetLag5']:
    # fit ARIMA on returns
    arima_model_fitted = pmdarima.auto_arima(daxdata[h], stationary=True)
    order_dict_aic.update({h: arima_model_fitted.order})

order_dict_aic

{'LogRetLag1': (0, 0, 0),
 'LogRetLag2': (5, 0, 0),
 'LogRetLag3': (5, 0, 1),
 'LogRetLag4': (5, 0, 0),
 'LogRetLag5': (5, 0, 2)}

In [28]:
order_dict_bic = {}
for h in ['LogRetLag1', 'LogRetLag2', 'LogRetLag3', 'LogRetLag4', 'LogRetLag5']:
    # fit ARIMA on returns
    arima_model_fitted = pmdarima.auto_arima(daxdata[h], stationary=True)
    order_dict_bic.update({h: arima_model_fitted.order})

order_dict_bic

{'LogRetLag1': (0, 0, 0),
 'LogRetLag2': (5, 0, 0),
 'LogRetLag3': (5, 0, 1),
 'LogRetLag4': (5, 0, 0),
 'LogRetLag5': (5, 0, 2)}

Glücklicherweise selbe order bei beiden Kriterien --> Nutze diese!  

## Volatility Component 
Find best p and q 

In [66]:
import pmdarima
import numpy as np
import arch

# Define a range for GARCH orders
p_values = range(1, 5)  # adjust as needed
q_values = range(1, 5)  # adjust as needed

best_orders_aic = {}
best_orders_bic = {}

for h in [1,2,3,4,5]:
    # ARIMA model
    temp_model = pmdarima.auto_arima(daxdata.iloc[:,h], suppress_warnings=True)
    temp_residuals = temp_model.arima_res_.resid

    # GARCH model
    garch_temp = arch.arch_model( 
        temp_residuals, p=1, q=1, dist='t')
    garch_fitted = garch_temp.fit(disp='off')

    # Forecasting
    predicted_mu = temp_model.predict(n_periods=h).iloc[-1]
    garch_forecast = garch_fitted.forecast(horizon=h)
    horizon_name = f'h.{h}'
    predicted_et = garch_forecast.mean[horizon_name].iloc[-1]

    # Final prediction
    prediction = predicted_mu + predicted_et

    print(f"{h} - ARIMA Order: {temp_model.order}, GARCH Order: (1, 1)")
    print("Final Prediction:", prediction)

c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


1 - ARIMA Order: (0, 0, 0), GARCH Order: (1, 1)
Final Prediction: 0.09044223109790384


c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


2 - ARIMA Order: (5, 0, 0), GARCH Order: (1, 1)
Final Prediction: 0.029662790098789985


c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


3 - ARIMA Order: (5, 0, 1), GARCH Order: (1, 1)
Final Prediction: -0.10087012353997829


c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


4 - ARIMA Order: (5, 0, 0), GARCH Order: (1, 1)
Final Prediction: 0.07400974046221918
5 - ARIMA Order: (5, 0, 2), GARCH Order: (1, 1)
Final Prediction: 0.35043469800250454


c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [64]:
temp_model.predict(n_periods=h)

c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


1104    0.783102
1105    0.718666
1106    0.665614
1107    0.817193
1108    0.244860
dtype: float64

Alternative GARCH Model with optimized GARCH Parameters und mean=zero

In [81]:
import arch
# Define a range for GARCH orders
p_values = range(1, 5)  # adjust as needed
q_values = range(1, 5)  # adjust as needed
lag_values = range(0,5)

best_orders_aic={}
best_orders_bic={}

for h in ['LogRetLag1', 'LogRetLag2', 'LogRetLag3', 'LogRetLag4','LogRetLag5']:
    # Grid search for best GARCH orders using AIC
   
    best_aic = float("inf")
    best_order = None

    for p in p_values:
        for q in q_values:
            for lag in lag_values: 
                model = arch.arch_model(daxdata[h], mean='AR', lags = lag, vol='Garch', p=p, q=q, dist='t')
                result = model.fit(disp='off')
                aic = result.aic
                if aic < best_aic:
                    best_aic = aic
                    best_order = (p, q)
                    best_lag = lag
    best_orders_aic.update({h: [best_aic, best_order, lag]})

    best_bic = float("inf")
    best_order = None

    for p in p_values:
        for q in q_values:
            for lag in lag_values: 
                model = arch.arch_model(daxdata[h], mean='AR', lags = lag, vol='Garch', p=p, q=q, dist='t')
                result = model.fit(disp='off')
                bic = result.bic
                if bic < best_bic:
                    best_bic = bic
                    best_order = (p, q)
                    best_lag = lag
    best_orders_bic.update({h: [best_bic, best_order, lag]})

In [80]:
best_orders_aic

{'LogRetLag1': [3299.0289013353513, (1, 1), 4],
 'LogRetLag2': [3518.310435423856, (1, 1), 4],
 'LogRetLag3': [3677.9638894156005, (2, 1), 4],
 'LogRetLag4': [3842.902122610829, (4, 1), 4],
 'LogRetLag5': [3877.574033527474, (4, 1), 4]}

In [82]:
best_orders_bic

{'LogRetLag1': [3335.835347361089, (1, 1), 4],
 'LogRetLag2': [3563.338024552934, (1, 1), 4],
 'LogRetLag3': [3724.33194858793, (1, 1), 4],
 'LogRetLag4': [3899.342676846499, (2, 1), 4],
 'LogRetLag5': [3929.615072402873, (1, 1), 4]}

In [13]:
best_orders_aic # with mean zero

{'LogRetLag1': [28525.96408653065, (2, 1)],
 'LogRetLag2': [34446.041667757985, (1, 4)],
 'LogRetLag3': [37501.865788263414, (1, 4)],
 'LogRetLag4': [39473.77774045375, (1, 4)],
 'LogRetLag5': [41160.60766632941, (1, 4)]}

In [14]:
best_orders_aic

{'LogRetLag1': [28488.880522565232, (3, 4)],
 'LogRetLag2': [34396.22638366342, (1, 4)],
 'LogRetLag3': [37452.05050416885, (1, 4)],
 'LogRetLag4': [39423.962456359186, (1, 4)],
 'LogRetLag5': [41110.792382234846, (1, 4)]}

In [75]:
# Prediction with GARCH(1,1) and mean zero 
import arch
from dax.help_functions.get_quantiles import get_t_quantiles

horizon_estimates ={}

for h in [1,2,3,4,5]:
    # Grid search for best GARCH orders using AIC
    model = arch.arch_model(
        daxdata.iloc[:,h], mean='zero', vol='Garch', p=1, q=1, dist='t')
    result = model.fit(disp='off')

    # predict variance
    garch_forecast = result.forecast(horizon=h)
    horizon_name = f'h.{h}'
    variance_prediction = garch_forecast.variance[horizon_name].iloc[-1]
    df = int(result.params['nu'])

    # store data
    horizon_estimates[h] = (df,variance_prediction)

# get quantiles
quantiles = [get_t_quantiles(pair) for pair in horizon_estimates.values()]

# create submission frame
column_names = [f'q{q}' for q in [0.025, 0.25, 0.5, 0.75, 0.975]]
dates = next_working_days(max(daxdata.index), 5)
quantile_df = pd.DataFrame(quantiles, columns=column_names)
quantile_df['date_time'] = dates
quantile_df.set_index('date_time', inplace=True)
quantile_df




,q0.025,q0.25,q0.5,q0.75,q0.975
date_time,,,,,
2024-01-11,-2.044406,-0.545404,5.199669e-17,0.545404,2.044406
2024-01-12,-2.448383,-0.736331,7.122157e-17,0.736331,2.448383
2024-01-15,-3.767726,-1.104888,1.065392e-16,1.104888,3.767726
2024-01-16,-4.255448,-1.303550,1.263741e-16,1.303550,4.255448
2024-01-17,-4.322703,-1.412060,1.380163e-16,1.412060,4.322703


In [93]:
import pandas as pd
import numpy as np

import arch

from dax.models.combination.ARMA_GARCH.get_quantiles import get_t_quantiles
from evaluation.help_functions.prepare_data import next_working_days
from dax.help_functions.get_dax_data import get_prepared_data

# decided for BIC since models have less parameters (simplicity)
par_opt_zero_mean = {
    1: [2, 1],  # criterion: BIC
    2: [1, 4],  # criterion: AIC, BIC
    3: [1, 4],  # criterion: AIC, BIC
    4: [1, 4],  # criterion: AIC, BIC
    5: [1, 4],  # criterion: AIC, BIC
}

par_opt_ar_mean = {
    1: [1, 1, 4],  # criterion: AIC, BIC
    2: [1, 1, 4],  # criterion: AIC, BIC
    3: [1, 1, 4],  # criterion: BIC
    4: [2, 1, 4],  # criterion: BIC
    5: [1, 1, 4]   # criterion: BIC
}


def get_arma_garch_forecasts(daxdata=pd.DataFrame, quantiles=[0.025, 0.25, 0.5, 0.75, 0.975], basic=False, opt_pq=False, opt_lag_pq=False, submission=True):
    ''' ARMA-GARCH Models:
        * basic --> GARCH(1,1) with zero mean
        * opt_pq --> GARCH (p,q) with zero mean and opt. p,q values 
        * opt_lag_pq --> AR(lag)-GARCH (p,q) model with opt. values
    '''

    if daxdata.empty:
        daxdata = get_prepared_data()
    date_st = daxdata.index[-1].strftime('%Y-%m-%d')

    horizon_estimates = {}
    for h in [1, 2, 3, 4, 5]:

        if basic == True:
            model = arch.arch_model(
                daxdata.iloc[:, h], mean='zero', vol='Garch', p=1, q=1, dist='t')
            result = model.fit(disp='off')

        elif opt_pq == True:
            p, q = par_opt_zero_mean[h][0], par_opt_zero_mean[h][1]
            model = arch.arch_model(
                daxdata.iloc[:, h], mean='zero', vol='Garch', p=p, q=q, dist='t')
            result = model.fit(disp='off')

        elif opt_lag_pq == True:
            p, q, lag = par_opt_ar_mean[h][0], par_opt_ar_mean[h][1], par_opt_ar_mean[h][2]
            model = arch.arch_model(
                daxdata.iloc[:, h], mean='AR', lags=lag, vol='Garch', p=p, q=q, dist='t')
            result = model.fit(disp='off')

        # predict mean, variance and df
        garch_forecast = result.forecast(horizon=h)
        horizon_name = f'h.{h}'
        variance_prediction = garch_forecast.variance[horizon_name].iloc[-1]
        mean_prediction = garch_forecast.mean[horizon_name].iloc[-1]
        df = int(result.params['nu'])

        # store data
        horizon_estimates[h] = (df, variance_prediction, mean_prediction)

    # get quantiles
    quantiles_calc = [get_t_quantiles(tuple)
                 for tuple in horizon_estimates.values()]

    # create quantile frame
    column_names = [f'q{q}' for q in quantiles]
    dates = next_working_days(max(daxdata.index), 5)
    quantile_df = pd.DataFrame(quantiles_calc, columns=column_names)
    quantile_df['date_time'] = dates
    quantile_df.set_index('date_time', inplace=True)

    # create submission frame
    if submission == True:
        quantile_df.insert(0, 'forecast_date', date_st)
        quantile_df.insert(1, 'target', 'DAX')
        quantile_df.insert(
            2, "horizon", [str(i) + " day" for i in (1, 2, 5, 6, 7)])

    return quantile_df

In [94]:
get_arma_garch_forecasts(basic=True)

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
date_time,,,,,,,,
2024-01-11,2024-01-10,DAX,1 day,-2.044406,-0.545404,5.199669e-17,0.545404,2.044406
2024-01-12,2024-01-10,DAX,2 day,-2.448383,-0.736331,7.122157e-17,0.736331,2.448383
2024-01-15,2024-01-10,DAX,5 day,-3.767726,-1.104888,1.065392e-16,1.104888,3.767726
2024-01-16,2024-01-10,DAX,6 day,-4.255448,-1.303550,1.263741e-16,1.303550,4.255448
2024-01-17,2024-01-10,DAX,7 day,-4.322703,-1.412060,1.380163e-16,1.412060,4.322703


In [95]:
get_arma_garch_forecasts(opt_lag_pq=True)

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
date_time,,,,,,,,
2024-01-11,2024-01-10,DAX,1 day,-2.304876,-0.521275,0.043040,0.607355,2.390955
2024-01-12,2024-01-10,DAX,2 day,-2.651404,-0.608537,0.134750,0.878037,2.920904
2024-01-15,2024-01-10,DAX,5 day,-3.666530,-1.056335,-0.106629,0.843078,3.453272
2024-01-16,2024-01-10,DAX,6 day,-3.151653,-0.677367,0.349285,1.375936,3.850222
2024-01-17,2024-01-10,DAX,7 day,-3.703519,-0.615234,0.601871,1.818977,4.907262


In [96]:
get_arma_garch_forecasts(opt_pq=True)

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
date_time,,,,,,,,
2024-01-11,2024-01-10,DAX,1 day,-2.061167,-0.549876,5.242299e-17,0.549876,2.061167
2024-01-12,2024-01-10,DAX,2 day,-2.341367,-0.704147,6.810857e-17,0.704147,2.341367
2024-01-15,2024-01-10,DAX,5 day,-3.347284,-1.006669,9.736991e-17,1.006669,3.347284
2024-01-16,2024-01-10,DAX,6 day,-3.527705,-1.126053,1.097353e-16,1.126053,3.527705
2024-01-17,2024-01-10,DAX,7 day,-4.040315,-1.319815,1.290002e-16,1.319815,4.040315


In [40]:
arma_model = pmdarima.arima.ARIMA(order=(1,0,1))
arma_model.fit(daxdata['LogRetLag1'])


# Use ARMA to predict mu --> mean component
predicted_mu = arma_model.predict(n_periods=1)

# fit a GARCH(1,1) model on the residuals of the ARMA model
garch = arch.arch_model(arma_residuals, p=1, q=1)
garch_fitted = garch.fit()

# Use GARCH to predict the residual --> variance component 
garch_variance = garch_fitted.forecast(horizon=1)
predicted_et = garch_variance.mean['h.1'].iloc[-1]

print(garch_variance)
# Combine both models' output: yt = mu + et
prediction = predicted_mu + predicted_et
print(prediction)

c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Iteration:      1,   Func. Count:      6,   Neg. LLF: 21329.154582094467
Iteration:      2,   Func. Count:     15,   Neg. LLF: 401485083641.05475
Iteration:      3,   Func. Count:     23,   Neg. LLF: 2118.9392873926745
Iteration:      4,   Func. Count:     30,   Neg. LLF: 2151.5869944546585
Iteration:      5,   Func. Count:     36,   Neg. LLF: 1683.724922313428
Iteration:      6,   Func. Count:     42,   Neg. LLF: 1681.7348762545657
Iteration:      7,   Func. Count:     47,   Neg. LLF: 1681.7196036188511
Iteration:      8,   Func. Count:     52,   Neg. LLF: 1681.719385786248
Iteration:      9,   Func. Count:     57,   Neg. LLF: 1681.719354752845
Iteration:     10,   Func. Count:     62,   Neg. LLF: 1681.719340292042
Iteration:     11,   Func. Count:     66,   Neg. LLF: 1681.7193402909604
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1681.719340292042
            Iterations: 11
            Function evaluations: 66
            Gradient evaluati

c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\Maria\Documents\Setups\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [23]:
arima_residuals = arima_model.arima_res_.resid

# fit a GARCH(1,1) model on the residuals of the ARIMA model
garch = arch.arch_model(arima_residuals, p=1, q=1)
garch_fitted = garch.fit()

# Use ARIMA to predict mu
predicted_mu = arima_model.predict(n_periods=1)[0]
# Use GARCH to predict the residual
garch_forecast = garch_model.forecast(horizon=1)
predicted_et = garch_forecast.mean['h.1'].iloc[-1]
# Combine both models' output: yt = mu + et
prediction = predicted_mu + predicted_et

TypeError: ARIMA.fit() missing 1 required positional argument: 'y'

AR(2) passt noch am besten, aber alle Modelle schlecht --> trotzdem mal ausprobieren

In [19]:
from arch.univariate import ARX, GARCH

# mean component 
mean_model_ar2 = ARX(daxdata['LogRetLag1'], lags=[2])

# volatility component 
mean_model_ar2.volatility = GARCH(p=1, q=1)

result = mean_model_ar2.fit(update_freq=0, disp="off")

forecast = result.forecast(horizon=1, reindex=False)
forecast.variance

,h.1
2023-11-22,0.717912
